In [1]:
# Import required libraries
from openai import OpenAI
from dotenv import load_dotenv
from datetime import datetime
import sqlglot
from sqlglot.errors import ParseError
import os
import re

In [2]:
# Load environment variables and configure settings
load_dotenv()

# Configuration
api_key = os.getenv("OPENAI_API_KEY")
test_folder_path = os.getenv("TEST_FOLDER_PATH")
model = "gpt-4"
temperature = 0.2

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

print("✅ Configuration loaded successfully")
print(f"Model: {model}")
print(f"Test folder: {test_folder_path}")

✅ Configuration loaded successfully
Model: gpt-4
Test folder: tests/generated/


In [3]:
# Utility function to load context files
def load_file(filename):
    """Load content from a file."""
    try:
        with open(filename, "r") as f:
            content = f.read()
            print(f"✅ Loaded {filename}")
            return content
    except FileNotFoundError:
        print(f"⚠️ Warning: {filename} not found. Using empty content.")
        return ""

# Load database context and guidelines
db_context = load_file("db_description.txt")
guidelines = load_file("chatbot_guidelines.txt")

extract_command = '''
    Please take the following chat history which is ordered by time of occurrence and extract the SQL test command which you think
    the user most likely want to deploy. Return only this SQL in plain text without any surrounding text. The chat history is as follows: {0}
'''

✅ Loaded db_description.txt
✅ Loaded chatbot_guidelines.txt


In [4]:
# Initialize conversation history with system prompt
messages = [
    {
        "role": "system",
        "content": f"""
You are a helpful assistant that writes SQL data quality test queries.

Use this database context for your answers:
---
{db_context}
---
In case the user suggests that he wants to deploy the test or save the test, inform him that he needs to type 'save test' to save the test to the database.

Also use these guidelines:
---
{guidelines}
---
""",
    }
]

# Variable to store the last assistant reply for saving tests
last_assistant_reply = ""

print("✅ Conversation initialized")
print(f"System prompt configured with {len(db_context)} characters of DB context")
print(f"Guidelines configured with {len(guidelines)} characters")

✅ Conversation initialized
System prompt configured with 231 characters of DB context
Guidelines configured with 308 characters


In [5]:
# Helper functions for SQL extraction and file operations

def extract_sql_from_response(text):
    """Extract SQL code from a GPT response."""
    match = re.search(r"```sql(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()  # fallback

def save_test_to_file(test_sql, description):
    """Save a SQL test to a file with timestamp."""
    if not test_folder_path:
        raise ValueError("TEST_FOLDER_PATH not configured")
    
    # Create directory if it doesn't exist
    os.makedirs(test_folder_path, exist_ok=True)
    
    # Create a safe filename
    slug = re.sub(r"\W+", "_", description.lower())[:50]
    filename = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_{slug}.sql"
    filepath = os.path.join(test_folder_path, filename)

    with open(filepath, "w") as f:
        f.write(f"-- Test: {description}\n\n{test_sql.strip()}\n")

    print(f"✅ Test saved to {filepath}")
    return filepath

print("✅ Helper functions defined")

✅ Helper functions defined


In [6]:
# Main function to send messages to the chatbot
def send_message(user_input):
    """
    Send a message to the chatbot and get a response.
    
    Args:
        user_input: The user's message
        
    Returns:
        The assistant's response
    """
    global messages, last_assistant_reply
    
    # Handle save test command
    if user_input.lower() == "save test":
        sql_candidate = send_message(extract_command.format(messages))
        try: 
            parsed = sqlglot.parse_one(sql_candidate)
            filepath = save_test_to_file(sql_candidate, "save test")
            return f"Test saved to {filepath}"
        except ParseError as e:
            return "No previous test to save. Please generate a test first."
    
    # Add user message to history
    messages.append({"role": "user", "content": user_input})

    # Get response from OpenAI
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

    # Extract reply
    assistant_reply = response.choices[0].message.content
    last_assistant_reply = assistant_reply

    # Add assistant response to history
    messages.append({"role": "assistant", "content": assistant_reply})
    
    return assistant_reply

print("✅ Main send_message function defined")

✅ Main send_message function defined
